In [21]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
# from docplex.mp.model import Model
from gurobipy import GRB, Model

In [22]:
static = pd.read_csv('static_with_t.csv')
detected = pd.read_csv('randomized_graph.csv')

In [23]:
node_name = 'root_cross_left'

In [24]:
static_node = static[static['name'] == node_name].to_dict('records')[0]

In [25]:
static_node

{'name': 'root_cross_left',
 'number': -2,
 'x': -13.0,
 'y': -20.0,
 'z': 0.0,
 'parent': 'root',
 'distance_to_parent': 24}

In [26]:
detected_node = detected[detected['name'] == node_name].to_dict('records')[0]

In [27]:
# detected_node['x']=12

In [28]:
detected_node

{'name': 'root_cross_left',
 'number': -2,
 'x': -12.22,
 'y': -20.0,
 'z': 4.45,
 'parent': 'root'}

In [29]:
# quadratic_model = Model(name='quadratic')
quadratic_model = Model('quadratic')

In [30]:
# x = quadratic_model.continuous_var(name = 'x')
# y = quadratic_model.continuous_var(name = 'y')
# z = quadratic_model.continuous_var(name = 'z')

x = quadratic_model.addVar(name = 'x', vtype=GRB.CONTINUOUS)
y = quadratic_model.addVar(name = 'y', vtype=GRB.CONTINUOUS)
z = quadratic_model.addVar(name = 'z', vtype=GRB.CONTINUOUS)

In [31]:
# quadratic_model.add(
#     x**2 + y**2 + z**2 <= static_node["distance_to_parent"] ** 2
# )  # to dziala tylko dla tego noda bo parent ma pos 0,0,0

quadratic_model.addQConstr(
    x**2 + y**2 + z**2 <= static_node["distance_to_parent"] ** 2
)  # to dziala tylko dla tego noda bo parent ma pos 0,0,0

<gurobi.QConstr Not Yet Added>

In [32]:
obj_fn = (x-static_node["x"])**2 + (y-static_node["y"])**2 + (z-static_node["z"])**2

In [33]:
# quadratic_model.set_objective('min', obj_fn)

quadratic_model.setObjective(obj_fn, GRB.MINIMIZE)

In [34]:
# quadratic_model.print_information()

quadratic_model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11500H @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0x9d557401
Model has 3 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e+01, 4e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [6e+02, 6e+02]
Presolve time: 0.04s
Presolved: 8 rows, 9 columns, 12 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.400e+01
 Factor NZ  : 3.600e+01
 Factor Ops : 2.040e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       P

In [35]:
for v in quadratic_model.getVars():
    print('%s %g' % (v.varName, v.x))

x 7.96198e-11
y 5.1759e-11
z 0.000332386


In [36]:
calc_x = quadratic_model.getVarByName('x').x
calc_y = quadratic_model.getVarByName('y').x
calc_z = quadratic_model.getVarByName('z').x

In [37]:
f_x_y_z = np.sqrt((calc_x-static_node["x"])**2 + (calc_y-static_node["y"])**2 + (calc_z-static_node["z"])**2)

In [38]:
f_x_y_z

23.853720886155706

In [39]:
calc_distance = np.sqrt(calc_x**2 + calc_y**2 + calc_z**2)
calc_distance

0.0003323859407984909

In [40]:
print('Obj: %g' % np.sqrt(quadratic_model.objVal))

Obj: 23.8537
